In [2]:
import pandas as pd

In [3]:
base_url = "https://en.wikipedia.org/wiki/"
file = "Resources/cleaned_data/cleaned_billionaires.csv"

In [3]:
#grab billionaires 
bill_df = pd.read_csv(file)

In [4]:
names = bill_df['Name']
names = list(names)

In [5]:
names = [name.split(' &')[0] for name in names]
names = [name.replace(' ', '+') for name in names]
names[:5]

['Jeff+Bezos', 'Elon+Musk', 'Bernard+Arnault', 'Bill+Gates', 'Mark+Zuckerberg']

In [6]:
len(names)

2755

In [7]:
## trying a different method than just pd.read_html
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import unidecode
from pymongo import MongoClient

In [8]:
base_url = "https://www.google.com/search?q=wikipedia+billionaire+"
executable_path = {'executable_path':ChromeDriverManager().install()}

conn = 'mongodb://localhost:27017'
client = MongoClient(conn)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\JordanRoessle\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [9]:
# database setup
db = client.billionaire

In [10]:
%%timeit
list_of_tables = []
for count, name in enumerate(names):
    try:
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(base_url + name)
        browser.links.find_by_partial_text('Wikipedia').click()
        soup = BeautifulSoup(browser.html, "html.parser")
        table_html = soup.find("table", class_="infobox biography vcard")
        if not table_html:
            table_html = soup.find("table", class_="infobox vcard")
        
        # turn table into dataframe
        df = pd.read_html(str(table_html))[0]
        
        # drop null rows
        df.dropna(inplace = True)
        
        mongo_dict = {'name':name.replace('+', ' ')}
        for index, row in df.iterrows():
            #some keys have '.' and that doesn't work for keys. We will ignore those cases.
            key = unidecode.unidecode(row[0])
            if '.' not in key:
                value = unidecode.unidecode(row[1])
                mongo_dict[key] = value

        db.wikiTables.insert_one(mongo_dict)
        browser.quit()
    except Exception as e:
        browser.quit()
        print(count, '\n', name, '\n', e)
        

68 
 Jiang+Rensheng 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

81 
 Li+Xiting 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

91 
 Xu+Hang 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

104 
 Chen+Bang 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

120 
 Chen+Zhiping 
 No tables found
129 
 Zuo+Hui 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

162 
 Wang+Liping 
 No tables found
165 
 Huang+Shilin 
 No tables found
168 
 Dang+Yanbao 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

180 
 Yu+Renrong 
 No tables found
181 
 Liu+Yonghao 
 No tables found
188 
 Cen+Junda 
 No tables found
193 
 Jorge+Moll+Filho 
 Message: element not interactable
  (Session info: chrome=91.0.4472.101)

200 
 Andreas+Struengmann 
 No tables found
201 
 Thomas+Struengmann 
 No tables found
206 
 Wang+Laisheng 
 Message: element not inte

KeyboardInterrupt: 

In [11]:
# reset database
db.wikiTables.delete_many({})